# Chapter 0 - 安裝與設定

## Install

In [1]:
%%capture
!pip install -U crawl4ai
!pip install nest_asyncio

In [2]:
# Check crawl4ai version
import crawl4ai
print(crawl4ai.__version__.__version__)

0.6.3


## Setup

In [3]:
%%capture
!crawl4ai-setup

## Test

In [4]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check... 
[INIT].... → Crawl4AI 0.6.3 
[TEST].... ℹ Testing crawling capabilities... 
[EXPORT].. ℹ Exporting media (PDF/MHTML/screenshot) took 0.63s 
[FETCH]... ↓ https://crawl4ai.com                                               
| ✓ | ⏱: 4.36s 
[SCRAPE].. ◆ https://crawl4ai.com                                               
| ✓ | ⏱: 0.01s 
[COMPLETE] ● https://crawl4ai.com                                               
| ✓ | ⏱: 4.37s 
[COMPLETE] ● ✅ Crawling test passed! 


In [5]:
import asyncio # 導入Python的非同步程式設計標準庫
import nest_asyncio # 導入巢狀非同步事件迴圈支援庫
nest_asyncio.apply() # 允許在Jupyter中使用非同步操作

In [ ]:
from playwright.async_api import async_playwright

async def test_browser():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless = True)
        page = await browser.new_page()
        await page.goto('https://example.com')
        #await page.goto('https://www.chromium.org/Home/')
        print(f'Title: {await page.title()}')
        await browser.close()

asyncio.run(test_browser())

Title: Chromium


## *Markdown Output Function

In [11]:
import os

OUTPUT_PATH = 'outputs/markdown/'

def output_md(base_filename, md_str):
    # 建立輸出目錄
    os.makedirs(OUTPUT_PATH, exist_ok=True)

    # 產生帶長度的檔案名稱
    length = len(md_str)
    name, ext = os.path.splitext(base_filename)
    filename = f"{name}({length}){ext}"

    # 完整路徑
    full_path = os.path.join(OUTPUT_PATH, filename)

    with open(full_path, 'w', encoding='utf-8') as f:
        f.write(md_str)

    print(f"已儲存到: {full_path}")



# Chapter 1 - 基礎形態

## 1.1 - Basic Type

In [12]:
import asyncio  # 非同步程式設計庫
from crawl4ai import AsyncWebCrawler  # 網頁抓取工具

# 非同步抓取網頁內容
async def main(output_filename):
    # 建立爬蟲物件，自動管理資源(確保爬蟲使用完後會自動關閉，釋放資源)
    async with AsyncWebCrawler() as crawler:
        # 存取指定網址並等待回應(await 關鍵字表示等待這個操作完成後再繼續執行下面的程式碼)
        result = await crawler.arun("https://www.anthropic.com/news/agent-capabilities-api")

        # 列印抓取結果
        print("Markdown length:", len(result.markdown))
        print(result.markdown[:300])

        # 儲存到.md檔案
        output_md(output_filename, result.markdown)

# 啟動非同步程式
asyncio.run(main('1_1_Basic.md'))

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.anthropic.com/news/agent-capabilities-api                                                |
✓ | ⏱: 1.03s 

[SCRAPE].. ◆ https://www.anthropic.com/news/agent-capabilities-api                                                |
✓ | ⏱: 0.03s 

[COMPLETE] ● https://www.anthropic.com/news/agent-capabilities-api                                                |
✓ | ⏱: 1.06s 

Markdown length: 10941
[Skip to main content](https://www.anthropic.com/news/agent-capabilities-api#main-content)[Skip to footer](https://www.anthropic.com/news/agent-capabilities-api#footer)
[](https://www.anthropic.com/)
  * Claude
  * API
  * Solutions
  * Research
  * Commitments
  * Learn
[News](https://www.anthropic
已儲存到: outputs/markdown/1_1_Basic(10941).md


# Chapter 2 - 進階形態

## 2.1 - Setting with BrowerConfig（瀏覽器配置）

BrowserConfig - 控制瀏覽器本身的行為和啟動方式
- headless: 是否以無頭模式運行, 還是顯示完整介面
- user_agent: 設定使用者代理來模擬不同瀏覽器
- proxy_config: 配置代理伺服器等瀏覽器級別的設定
- text_mode: 禁用圖片載入，只抓取文本內容

In [13]:
import asyncio  # 非同步程式設計庫
from crawl4ai import AsyncWebCrawler, BrowserConfig
# AsyncWebCrawler: 非同步網頁爬蟲
# BrowserConfig: 瀏覽器配置
# CrawlerRunConfig: 爬蟲運行配置
# CacheMode: 快取模式控制

# 非同步主函數，執行網頁爬取任務
async def main(output_filename):
   # 配置瀏覽器參數
   browser_config = BrowserConfig(
       headless = True,  # 無頭模式，不顯示瀏覽器視窗
       viewport_width = 1280,   # 視窗寬度
       viewport_height = 720,   # 視窗高度
       user_agent = 'Chrome/114.0.0.0',  # 瀏覽器標識
       text_mode = True, #禁用圖片載入，可能會加速僅文本的爬取
   )

   # 建立非同步網頁爬蟲，自動管理資源
   async with AsyncWebCrawler(config = browser_config) as crawler:
       # 執行網頁爬取
        result = await crawler.arun(
            url = "https://www.anthropic.com/news/agent-capabilities-api",  # 目標網址
        )

        # 顯示爬取結果
        print("Markdown length:", len(result.markdown))  # 內容長度
        print(result.markdown[:300])  # 前300字元預覽

        output_md(output_filename, result.markdown)

# 啟動非同步程式
asyncio.run(main('2_1_BrowserConfig.md'))

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.anthropic.com/news/agent-capabilities-api                                                |
✓ | ⏱: 1.15s 

[SCRAPE].. ◆ https://www.anthropic.com/news/agent-capabilities-api                                                |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.anthropic.com/news/agent-capabilities-api                                                |
✓ | ⏱: 1.16s 

Markdown length: 10941
[Skip to main content](https://www.anthropic.com/news/agent-capabilities-api#main-content)[Skip to footer](https://www.anthropic.com/news/agent-capabilities-api#footer)
[](https://www.anthropic.com/)
  * Claude
  * API
  * Solutions
  * Research
  * Commitments
  * Learn
[News](https://www.anthropic
已儲存到: outputs/markdown/2_1_BrowserConfig(10941).md


## 2.2.0 - Setting with CrawlerRunConfig (爬蟲運行配置)

CrawlerRunConfig - 控制每次具體爬取任務的執行方式
- word_count_threshold: 過濾掉過短的內容，比如導覽選單、按鈕文字、簡短標籤
- extraction_strategy: 自訂抓取內容，需要定義json的schema
- cache_mode: 快取策略, 是否使用快取
- js_code: 模擬使用者點擊[Load More]等按鈕
- screenshot: 在頁面完全載入後自動截取網頁截圖
- pdf: 將整個網頁轉換為PDF文件
- [重要] markdown_generator: 預設DefaultMarkdownGenerator()

In [ ]:
import asyncio  # 非同步程式設計庫
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
# AsyncWebCrawler: 非同步網頁爬蟲
# BrowserConfig: 瀏覽器配置
# CrawlerRunConfig: 爬蟲運行配置
# CacheMode: 快取模式控制
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator

# 非同步主函數，執行網頁爬取任務
async def main(output_filename):
   # 配置瀏覽器參數
   browser_config = BrowserConfig(
       headless = True,  # 無頭模式，不顯示瀏覽器視窗
       viewport_width = 1280,   # 視窗寬度
       viewport_height = 720,   # 視窗高度
       user_agent = 'Chrome/114.0.0.0',  # 瀏覽器標識
       text_mode = True, #禁用圖片載入，可能會加速僅文本的爬取
   )

   # 配置爬蟲運行參數
   run_config = CrawlerRunConfig(
       cache_mode = CacheMode.DISABLED,  # 禁用快取，獲取最新內容
       markdown_generator = DefaultMarkdownGenerator(),
   )

   # 建立非同步網頁爬蟲，自動管理資源
   async with AsyncWebCrawler(config = browser_config) as crawler:
       # 執行網頁爬取
        result = await crawler.arun(
            url = "https://www.anthropic.com/news/agent-capabilities-api",  # 目標網址
            config = run_config,  # 運行配置
        )

        # 顯示爬取結果
        print("Markdown length:", len(result.markdown))  # 內容長度
        print(result.markdown[:300])  # 前300字元預覽

        output_md(output_filename, result.markdown)

# 啟動非同步程式
asyncio.run(main('2_2_0_RunConfig.md'))

### 2.2.1 + Content Filter: PruningContentFilter例

- **markdown_generator**: 核心功能，從網頁產生乾淨、結構化的Markdown
    - DefaultMarkdownGenerator(預設且唯一)
        - 參數1: Content Filters
            - BM25ContentFilter  關鍵詞過濾器
            - PruningContentFilter 內容精簡過濾器
            - LLMContentFilter AI過濾器

In [ ]:
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
from crawl4ai.content_filter_strategy import PruningContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator

async def main(output_filename):
    # 瀏覽器配置
    browser_config = BrowserConfig(headless = True, # 無頭模式
                                viewport_width = 1280,  # 視窗寬度
                                viewport_height = 720,  # 視窗高度
                                user_agent = 'Chrome/114.0.0.0', # 瀏覽器標識
                                text_mode = True,
                                 )

    # 爬蟲運行配置
    run_config = CrawlerRunConfig(
    cache_mode = CacheMode.DISABLED,  # 禁用快取
    markdown_generator = DefaultMarkdownGenerator(
        content_filter = PruningContentFilter(
            # min_word_threshold = 10, # 丟棄少於N個單詞的塊，因為它們可能太短或無用(不建議)
            threshold = 0.76,  # fixded: 固定閾值 / dynamic: 初始閾值
            threshold_type = "fixed", # 固定
            # threshold_type = "dynamic", # 變動
        )),
    )

    # 建立爬蟲並執行
    async with AsyncWebCrawler(config = browser_config) as crawler:
        result = await crawler.arun(
            url = "https://www.anthropic.com/news/agent-capabilities-api",  # 目標網址
            config = run_config,  # 運行配置
        )

        # 儲存原始內容
        print("Raw Markdown length:", len(result.markdown.raw_markdown))
        output_md(output_filename.replace('.md', '_raw.md'), result.markdown.raw_markdown)

        # 儲存過濾後內容
        print("Fit Markdown length:", len(result.markdown.fit_markdown))
        output_md(output_filename.replace('.md', '_fit.md'), result.markdown.fit_markdown)

asyncio.run(main('2_2_1_RunConfig_ContentFilterPruning.md'))

### 2.2.2 + Options

- **markdown_generator**: 核心功能，從網頁產生乾淨、結構化的Markdown
    - DefaultMarkdownGenerator(預設且唯一)
        - 參數1: Content Filters
            - BM25ContentFilter  關鍵詞過濾器
            - PruningContentFilter 內容精簡過濾器
            - LLMContentFilter AI過濾器
        - 參數2: Options
            - ignore_links (bool): 是否在最終markdown中移除所有超連結
            - ignore_images (bool): 移除所有 [[image]]() 圖片引用
            - escape_html (bool): 將HTML實體轉換為文本（預設通常為 True）
            - body_width (int): 在N個字元處換行。0 或 None 表示不換行
            - skip_internal_links (bool): 如果為 True，忽略 #localAnchors 或引用同一頁面的內部連結
            - include_sup_sub (bool): 嘗試以更易讀的方式處理 <sup> / <sub> 標籤

In [ ]:
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
from crawl4ai.content_filter_strategy import PruningContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator

async def main(output_filename):
    # 瀏覽器配置
    browser_config = BrowserConfig(headless = True, # 無頭模式
                                viewport_width = 1280,  # 視窗寬度
                                viewport_height = 720,  # 視窗高度
                                user_agent = 'Chrome/114.0.0.0', # 瀏覽器標識
                                text_mode = True,
                                 )

    # 爬蟲運行配置
    run_config = CrawlerRunConfig(
    cache_mode = CacheMode.DISABLED,  # 禁用快取
    markdown_generator = DefaultMarkdownGenerator(
        content_filter = PruningContentFilter(
            # min_word_threshold = 10, # 丟棄少於N個單詞的塊，因為它們可能太短或無用(不建議)
            threshold = 0.76,  # fixded: 固定閾值 / dynamic: 初始閾值
            # threshold_type = "fixed", # 固定
            threshold_type = "dynamic", # 變動
        ),
        options = {
            "ignore_links": True,
            "ignore_images": True,
            })
    )

    # 建立爬蟲並執行
    async with AsyncWebCrawler(config=browser_config) as crawler:
        result = await crawler.arun(
            url = "https://www.anthropic.com/news/agent-capabilities-api",  # 目標網址
            config = run_config,  # 運行配置
        )

        # 儲存原始內容
        print("Raw Markdown length:", len(result.markdown.raw_markdown))
        output_md(output_filename.replace('.md', '_raw.md'), result.markdown.raw_markdown)

        # 儲存過濾後內容
        print("Fit Markdown length:", len(result.markdown.fit_markdown))
        output_md(output_filename.replace('.md', '_fit.md'), result.markdown.fit_markdown)

asyncio.run(main('2_2_2_RunConfig_ContentFilterPruning_Options.md'))